# Diffusion Model Project

In [ ]:
from functions import *

In [ ]:
import torch
import diffusers
from PIL import Image
from tqdm import tqdm
import os

In [ ]:
# setup
model_id = "google/ddpm-bedroom-256" # "google/ddpm-celebahq-256"
model = diffusers.UNet2DModel.from_pretrained(model_id)
ddpm_scheduler = diffusers.DDPMScheduler.from_pretrained(model_id)
ddpm_scheduler.set_timesteps(50)

# Denoising Animation

In [ ]:
# input prepraration
image_size = model.config.sample_size # get image size
noise = torch.randn((1, 3, image_size, image_size)) # sample random noise

In [ ]:
# output generation
current = noise
history = [noise]
for i, t in enumerate(ddpm_scheduler.timesteps):
    with torch.no_grad():
        predicted_noise = model(current, t).sample
        current = ddpm_scheduler.step(predicted_noise, t, current).prev_sample
        show_table([[tensor_as_html(history[-1]), tensor_as_html(predicted_noise), tensor_as_html(current)], ["Image", "Predicted Noise", "Image - Predicted Noise"], ["", f"{i + 1}/50", ""]])
        history.append(current)

In [ ]:
# show images
show_images(*history[::10])

In [ ]:
# save image
os.makedirs("../output", exist_ok=True)
tensor_as_image(current).save("../output/output.png")

# Forward and Backward Process

In [ ]:
# forward process
noised_image = ddpm_scheduler.add_noise(current, noise, ddpm_scheduler.timesteps[30])
tensor_as_image(noised_image)

In [ ]:
# backward process
current = noised_image
for t in tqdm(ddpm_scheduler.timesteps[30:]):
    with torch.no_grad():
        predicted_noise = model(current, t).sample
        current = ddpm_scheduler.step(predicted_noise, t, current).prev_sample
        show_images(current)

In [ ]:
# compare images
show_images(history[-1], current, history[-1] - current)

In [ ]:
# create forward / backward process image
image = current # image_as_tensor(Image.open("../output/ddpm_9.png"))

# setup output image
output = Image.new("RGB", (image_size * 50, image_size * 50), (255, 255, 255))
output.paste(tensor_as_image(image), (0, 0))

# output generation
with tqdm(total=sum(range(1, 51))) as tqdm_bar:
    for i in reversed(range(50)):
        current = ddpm_scheduler.add_noise(image, noise, ddpm_scheduler.timesteps[i])
        output.paste(tensor_as_image(current), (0, image_size * (50 - i)))
        for j, t in enumerate(ddpm_scheduler.timesteps[i:]):
            with torch.no_grad():
                predicted_noise = model(current, t).sample
                current = ddpm_scheduler.step(predicted_noise, t, current).prev_sample
                output.paste(tensor_as_image(current), (image_size * (j + 1), image_size * (50 - i)))
            tqdm_bar.update(1)

# save output image
output.save("output/process.png")

# DDIM with Similar Noise

In [ ]:
# setup
ddim_scheduler = diffusers.DDIMScheduler.from_pretrained(model_id)
ddim_scheduler.set_timesteps(50)

In [ ]:
# prepare input
noise = torch.randn((1, 3, image_size, image_size)) # sample random noise
noises = [noise.clone() for _ in range(2)] # duplicate noise
noises[1][:,:,50:100,50:100] = torch.randn((1, 3, 50, 50)) # modify second noise
show_images(*noises)

In [ ]:
from time import sleep
for i in range(10):
    show_images(noises[i % 2])
    sleep(0.5)

In [ ]:
# output generation
images = list()
for current in noises:
    for t in tqdm(ddim_scheduler.timesteps):
        with torch.no_grad():
            predicted_noise = model(current, t).sample
            current = ddim_scheduler.step(predicted_noise, t, current).prev_sample
    images.append(current)

In [ ]:
# show output
show_images(*images)

In [ ]:
# save input and output
for i in range(len(noises)):
    tensor_as_image(noises[i]).save(f"../output/similar_ddim_noise_{i}.png")
    tensor_as_image(images[i]).save(f"../output/similar_ddim_image_{i}.png")

# DDIM Accelerated Sampling

In [ ]:
# output generation
current = noise
for t in tqdm(ddim_scheduler.timesteps):
    with torch.no_grad():
        predicted_noise = model(current, t).sample
        current = ddim_scheduler.step(predicted_noise, t, current).prev_sample
        show_images(current)

In [ ]:
ddim_scheduler.set_timesteps(1000)

In [ ]:
# output generation
second = noise
for t in tqdm(ddim_scheduler.timesteps):
    with torch.no_grad():
        predicted_noise = model(second, t).sample
        second = ddim_scheduler.step(predicted_noise, t, second).prev_sample
        show_images(second)

In [ ]:
show_images(current, second)